In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json
from tqdm.notebook import tqdm

/lustre/home/fdorner/miniforge3/envs/rejection-sampling-experiments/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from tqdm import tqdm

In [3]:
from accelerate import infer_auto_device_map

In [4]:
tokenizer = AutoTokenizer.from_pretrained("/fast/groups/sf/huggingface-models/meta-llama--Meta-Llama-3-70B")


In [5]:
model = AutoModelForCausalLM.from_pretrained("/fast/groups/sf/huggingface-models/meta-llama--Meta-Llama-3-70B",torch_dtype=torch.bfloat16,device_map="auto")

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 30/30 [00:53<00:00,  1.78s/it]


In [6]:
tokenizer.pad_token = tokenizer.eos_token

In [7]:
with open("full_prompts.json", "r", encoding="utf-8") as f:
    texts = json.load(f)

In [8]:
def difficulty_assessment_prompt(text):
    return "You will be shown a multiple choice question. Please rate its difficulty from 1 to 10. Questions of difficulty 1 should be easy for most kids in elementary school, while difficulty 10 represents PhD-level questions. \n" + text + "Difficulty estimate: " 

In [9]:
questions = [difficulty_assessment_prompt("Question: " +text.split("Question: ")[-1].split("Answer:")[0]) for text in texts]

In [10]:
print(questions[0])

You will be shown a multiple choice question. Please rate its difficulty from 1 to 10. Questions of difficulty 1 should be easy for most kids in elementary school, while difficulty 10 represents PhD-level questions. 
Question: Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.
A. 0
B. 4
C. 2
D. 6
Difficulty estimate: 


In [11]:
answer_tokens_llama = {key:tokenizer.encode(str(key))[-1] for key in range(1,11)}

In [12]:
softmax = torch.nn.Softmax(-1)

In [13]:
tokenizer.encode(" C")

[128000, 356]

In [14]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Sun Feb  1 14:15:52 2026       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:07:00.0 Off |                    0 |
| N/A   31C    P0              65W / 400W |  17117MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [15]:
batch_size = 5
diffs = []
for i in tqdm(range(0, len(questions), batch_size)):  
    encoded_input = tokenizer(questions[i:i+batch_size], return_tensors='pt',padding=True)

    shape = encoded_input["input_ids"].shape
    print(shape)

    output = model(**encoded_input)
    

    for i,n in enumerate(encoded_input.attention_mask.sum(-1)):
        #print(torch.argmax(output.logits[i][n-1]))
        subselect = softmax(output.logits[i][n-1])[[16, 17, 18, 19, 20, 21, 22, 23, 24, 605]]
        subselect = subselect/subselect.sum()
        diffs.append(subselect.argmax().detach().item()+1)

        

  0%|                                                                                           | 0/2809 [00:00<?, ?it/s]

torch.Size([5, 139])


  0%|                                                                                 | 1/2809 [00:02<1:54:07,  2.44s/it]

torch.Size([5, 134])


  0%|                                                                                 | 2/2809 [00:04<1:32:34,  1.98s/it]

...

100%|███████████████████████████████████████████████████████████████████████████████▉| 2807/2809 [41:14<00:01,  1.92it/s]

torch.Size([5, 96])


100%|███████████████████████████████████████████████████████████████████████████████▉| 2808/2809 [41:15<00:00,  1.96it/s]

torch.Size([2, 91])


100%|████████████████████████████████████████████████████████████████████████████████| 2809/2809 [41:15<00:00,  1.13it/s]


In [21]:
import numpy as np
for i in range(1,11):
    print((np.array(diffs)==i).sum())

8082
1
5
0
4472
5
443
278
0
756


In [22]:
np.save("Data/difficulty_llama70b.npy",diffs)